<a href="https://colab.research.google.com/github/cyamize/LDA/blob/main/LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import urllib.request
urllib.request.urlretrieve("https://raw.githubusercontent.com/franciscadias/data/master/abcnews-date-text.csv", filename="abcnews-date-text.csv")
data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False)

In [6]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
print(len(data))

1082168


In [3]:
data.head()

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers


In [4]:
text = data[['headline_text']]
text.head()

,headline_text
0,aba decides against community broadcasting lic...
1,act fire witnesses must be aware of defamation
2,a g calls for infrastructure protection summit
3,air nz staff in aust strike for pay rise
4,air nz strike to affect australian travellers


In [9]:
import nltk

text['headline_text'] = text.apply(lambda row: nltk.word_tokenize(row['headline_text']), axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
print(text.head())

                                       headline_text
0  [aba, decides, against, community, broadcastin...
1  [act, fire, witnesses, must, be, aware, of, de...
2  [a, g, calls, for, infrastructure, protection,...
3  [air, nz, staff, in, aust, strike, for, pay, r...
4  [air, nz, strike, to, affect, australian, trav...


In [12]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [17]:
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [14]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

text['headline_text'] = text['headline_text'].apply(lambda x : [word for word in x if word not in (stop)])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [15]:
print(text.head())

                                       headline_text
0   [aba, decides, community, broadcasting, licence]
1    [act, fire, witnesses, must, aware, defamation]
2     [g, calls, infrastructure, protection, summit]
3          [air, nz, staff, aust, strike, pay, rise]
4  [air, nz, strike, affect, australian, travellers]


In [19]:
from nltk.stem import WordNetLemmatizer
text['headline_text'] = text['headline_text'],apply(lambda x: [WordNetLemmatizer().lemmatizer(word, pos, ='v') for word in x])

int(text.head())

SyntaxError: ignored

In [20]:
tokenized_doc = text['headline_text'].apply(lambda x: [word for word in x if len(word)>3])
print(tokenized_doc[:5])

0    [decides, community, broadcasting, licence]
1     [fire, witnesses, must, aware, defamation]
2    [calls, infrastructure, protection, summit]
3                    [staff, aust, strike, rise]
4       [strike, affect, australian, travellers]
Name: headline_text, dtype: object


In [21]:
# TF-IDF

detokenized_doc = [ ]
for i in range(len(text)):
  t = ' '.join(tokenized_doc[i])
  detokenized_doc.append(t)

text['headline_text'] = detokenized_doc

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [22]:
text['headline_text'][:5]

0    decides community broadcasting licence
1      fire witnesses must aware defamation
2    calls infrastructure protection summit
3                    staff aust strike rise
4       strike affect australian travellers
Name: headline_text, dtype: object

In [23]:
text['headline_text'].head()

0    decides community broadcasting licence
1      fire witnesses must aware defamation
2    calls infrastructure protection summit
3                    staff aust strike rise
4       strike affect australian travellers
Name: headline_text, dtype: object

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(stop_words = 'english', max_features = 1000)
X = vect.fit_transform(text['headline_text'])
X.shape

(1082168, 1000)

In [29]:
# topic modeling

from sklearn.decomposition import LatentDirichletAllocation
lda_model = LatentDirichletAllocation(n_components=10, learning_method = 'online', random_state=777, max_iter= 1)

In [30]:
lda_top=lda_model.fit_transform(X)

In [31]:
print(lda_model.components_)
print(lda_model.components_.shape)

[[1.00000524e-01 1.00000945e-01 1.00003031e-01 ... 1.00010614e-01
  1.00003498e-01 1.00001781e-01]
 [1.00003244e-01 1.00006985e-01 1.00018485e-01 ... 1.00009374e-01
  1.00004140e-01 1.00002710e-01]
 [1.00000613e-01 1.00001003e-01 1.00001719e-01 ... 1.00010032e-01
  1.00003629e-01 1.00002942e-01]
 ...
 [1.00002359e-01 1.00002223e-01 1.00008046e-01 ... 2.25245378e+03
  1.00004394e-01 1.00001389e-01]
 [1.00001729e-01 1.00001155e-01 3.56084764e+03 ... 1.00006474e-01
  1.00003539e-01 1.00001515e-01]
 [1.00001008e-01 1.00001333e-01 1.00004934e-01 ... 1.00015214e-01
  1.00003583e-01 1.00001347e-01]]
(10, 1000)


In [34]:
terms = vect.get_feature_names()

def get_topics(components, feature_names, n=5):
  for i, topic in enumerate(components):
    print('Topic %d:'% (i+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n-1:--1]])

get_topics(lda_model.components_, terms)

Topic 1: [('podcast', 0.1), ('monday', 0.1), ('robbery', 0.1), ('2013', 0.1), ('thursday', 0.1), ('rudd', 0.1), ('wheat', 0.1), ('hour', 0.1), ('knights', 0.1), ('tsunami', 0.1), ('springs', 0.1), ('2014', 0.1), ('bashing', 0.1), ('gillard', 0.1), ('alice', 0.1), ('england', 0.1), ('friday', 0.1), ('custody', 0.1), ('tuesday', 0.1), ('abbott', 0.1), ('sentenced', 0.1), ('domestic', 0.1), ('quarter', 0.1), ('interview', 0.1), ('tony', 0.1), ('brown', 0.1), ('july', 0.1), ('bulldogs', 0.1), ('zimbabwe', 0.1), ('resources', 0.1), ('broncos', 0.1), ('sentence', 0.1), ('grandstand', 0.1), ('ashes', 0.1), ('mackay', 0.1), ('fishing', 0.1), ('grand', 0.1), ('carbon', 0.1), ('rural', 0.1), ('howard', 0.1), ('beef', 0.1), ('teacher', 0.1), ('speaks', 0.1), ('revamp', 0.1), ('lanka', 0.1), ('wallabies', 0.1), ('awards', 0.1), ('contract', 0.1), ('reporter', 0.1), ('hand', 0.1), ('pair', 0.1), ('fatal', 0.1), ('investigate', 0.1), ('candidate', 0.1), ('commission', 0.1), ('consider', 0.1), ('wrap

In [35]:
!pip install pyldavis

     |████████████████████████████████| 1.7MB 4.0MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 9.9MB 30.3MB/s 
     |████████████████████████████████| 15.7MB 13.5MB/s 
  Created wheel for pyldavis: filename=pyLDAvis-3.3.1-cp37-none-any.whl size=136897 sha256=5f8e5abf6f3238e8af927c78937c9196983e1d5e370d0e4882fff308c72feb4e
  Stored in directory: /root/.cache/pip/wheels/a0/9c/fc/c6e00689d35c82cf96a8adc70edfe7ba7904374fdac3240ac2
Successfully built pyldavis
ERROR: tensorflow 2.5.0 has requirement numpy~=1.19.2, but you'll have numpy 1.21.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement pandas~=1.1.0; python_version >= "3.0", but you'll have pandas 1.2.5 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 